In [1]:
import numpy as np
import pandas as pd
import os
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
path = "input/"

# original data : data 
data =  pd.read_csv(path + "demential_data.csv", parse_dates=['summary_date']) # 애초에 datatime 유형의 데이터를 읽어 올 때부터 형변환하기
data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,...,sleep_temperature_deviation,sleep_temperature_trend_deviation,timezone,sleep_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,...,14346,0,417,0,...,-0.12,99.99,NaN,\r,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,...,6352,0,473,0,...,-0.32,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,...,7297,0,586,0,...,0.07,99.99,NaN,\r,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,...,491,0,176,0,...,-0.41,99.99,NaN,\r,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,...,5861,0,646,0,...,-0.27,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,...,3863,3,735,2,...,-0.16,99.99,NaN,\r,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,...,4411,1,780,4,...,-0.09,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,...,11057,1,518,1,...,0.41,99.99,NaN,\r,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,...,5135,4,578,0,...,0.27,99.99,NaN,\r,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [3]:
# column의 수가 많기 때문에 효율적으로 관리하기 위해서 
# column의 이름들을 묶어서 진행합시다

drop_cols = ['timezone', 'EMAIL',
             'activity_class_5min', 'activity_met_1min', 
             'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min',
             'sleep_is_longest', 'sleep_temperature_trend_deviation', 'sleep_total', 'sleep_temperature_deviation']
convert_cols = data.iloc[:, 60:66].columns
data_convert = data[convert_cols].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [4]:
signal_list = list(data_convert['CONVERT(activity_class_5min USING utf8)'].str.split('/'))
signal_activity_class_5min = pd.DataFrame(signal_list).drop(288, axis=1).astype('float')
y_df = data_convert['DIAG_NM'] 
signal_activity_class_5min = pd.concat([signal_activity_class_5min, y_df], axis=1)
signal_activity_class_5min

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,DIAG_NM
0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
3,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9323,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9324,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9325,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN


In [5]:
signal_list = list(data_convert['CONVERT(activity_met_1min USING utf8)'].str.split('/'))
signal_activity_met_1min = pd.DataFrame(signal_list).drop(1440, axis=1).astype('float')
signal_activity_met_1min = pd.concat([signal_activity_met_1min, y_df], axis=1)
signal_activity_met_1min

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,DIAG_NM
0,0.9,0.9,1.4,1.9,1.1,0.9,0.9,1.1,1.3,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
2,1.1,1.1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.1,1.4,1.2,0.9,1.2,1.1,0.9,0.9,0.9,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9323,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9324,0.9,0.9,0.9,0.9,1.1,0.9,0.9,1.1,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9325,1.1,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN


In [6]:
signal_list = list(data_convert['CONVERT(sleep_hr_5min USING utf8)'].str.split('/'))
signal_sleep_hr_5min = pd.DataFrame(signal_list)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,0,73,73,73,72,71,70,71,71,71,...,None,None,None,None,None,None,None,None,None,None
1,69,70,69,69,70,72,71,72,70,69,...,None,None,None,None,None,None,None,None,None,None
2,0,74,73,73,74,74,74,71,71,70,...,None,None,None,None,None,None,None,None,None,None
3,73,70,71,72,75,75,73,70,70,70,...,None,None,None,None,None,None,None,None,None,None
4,0,0,0,0,0,0,0,0,69,69,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0,54,54,54,55,56,0,0,55,52,...,None,None,None,None,None,None,None,None,None,None
9323,68,66,67,67,68,69,69,70,71,71,...,None,None,None,None,None,None,None,None,None,None
9324,0,0,65,65,66,67,69,72,73,72,...,None,None,None,None,None,None,None,None,None,None
9325,0,0,0,64,64,64,64,65,65,66,...,None,None,None,None,None,None,None,None,None,None


In [7]:
signal_sleep_hr_5min = signal_sleep_hr_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hr_5min = signal_sleep_hr_5min.fillna(0) 
signal_sleep_hr_5min = signal_sleep_hr_5min.replace('', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.replace(' ', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.astype('float')

In [8]:
signal_sleep_hr_5min  = signal_sleep_hr_5min.transpose()
signal_sleep_hr_5min = pd.concat([signal_sleep_hr_5min, y_df], axis=1)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,73.0,73.0,73.0,72.0,71.0,70.0,71.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,69.0,70.0,69.0,69.0,70.0,72.0,71.0,72.0,70.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,74.0,73.0,73.0,74.0,74.0,74.0,71.0,71.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,73.0,70.0,71.0,72.0,75.0,75.0,73.0,70.0,70.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,54.0,54.0,54.0,55.0,56.0,0.0,0.0,55.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,68.0,66.0,67.0,67.0,68.0,69.0,69.0,70.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,65.0,65.0,66.0,67.0,69.0,72.0,73.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,64.0,64.0,64.0,64.0,65.0,65.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [9]:
signal_list = list(data_convert['CONVERT(sleep_hypnogram_5min USING utf8)'].str.split('/'))
signal_sleep_hypnogram_5min = pd.DataFrame(signal_list)

signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.drop(180, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.fillna(0) 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace('', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace(' ', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.astype('float')

In [10]:
signal_sleep_hypnogram_5min  = signal_sleep_hypnogram_5min.transpose()
signal_sleep_hypnogram_5min = pd.concat([signal_sleep_hypnogram_5min, y_df], axis=1)
signal_sleep_hypnogram_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,4.0,2.0,4.0,3.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,2.0,4.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,4.0,2.0,4.0,4.0,1.0,1.0,1.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,4.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,4.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,4.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [11]:
signal_list = list(data_convert['CONVERT(sleep_rmssd_5min USING utf8)'].str.split('/'))
signal_sleep_rmssd_5min = pd.DataFrame(signal_list)

signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.fillna(0) 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace('', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace(' ', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.astype('float')

In [12]:
signal_sleep_rmssd_5min  = signal_sleep_rmssd_5min.transpose()
signal_sleep_rmssd_5min = pd.concat([signal_sleep_rmssd_5min, y_df], axis=1)
signal_sleep_rmssd_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,10.0,10.0,10.0,11.0,11.0,10.0,12.0,18.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,23.0,23.0,26.0,24.0,18.0,13.0,15.0,14.0,17.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,11.0,14.0,20.0,13.0,14.0,14.0,16.0,27.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,24.0,28.0,19.0,17.0,12.0,10.0,17.0,20.0,23.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,35.0,39.0,28.0,26.0,41.0,0.0,0.0,52.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,26.0,16.0,19.0,18.0,16.0,18.0,19.0,17.0,15.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,13.0,12.0,12.0,13.0,12.0,12.0,12.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,14.0,17.0,18.0,14.0,14.0,17.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [13]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# y 변수와 X 변수 분할 관리 
input_data_y = y_df.copy()
input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X2 = signal_activity_met_1min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X3 = signal_sleep_hr_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X4 = signal_sleep_hypnogram_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X5 = signal_sleep_rmssd_5min.drop(labels = 'DIAG_NM', axis = 1)

In [14]:
# LabelEncoder
# y data를 LabelEncdoer로 한 번 더 labeling 합니다 -> 100만원, 10000만원... = 0, 1, ... 
# 추후에 DL에서 output을 맞춰주기 위함입니다, 최종 마지막에서 원래 label값(100만원, 1000만원...)으로 되돌립니다
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
input_data_y_label = le.fit_transform(list(input_data_y))

print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(input_data_y))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {'CN', 'MCI', 'Dem'}


In [42]:
X1_train_Label, X1_test_Label, y1_train_Label, y1_test_Label = train_test_split(input_data_X1, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X2_train_Label, X2_test_Label, y2_train_Label, y2_test_Label = train_test_split(input_data_X2, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X3_train_Label, X3_test_Label, y3_train_Label, y3_test_Label = train_test_split(input_data_X3, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X4_train_Label, X4_test_Label, y4_train_Label, y4_test_Label = train_test_split(input_data_X4, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X5_train_Label, X5_test_Label, y5_train_Label, y5_test_Label = train_test_split(input_data_X5, input_data_y_label, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)

In [16]:
import math
import sklearn
import sklearn.preprocessing
import datetime
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

#display parent directory and working directory
print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
print(os.getcwd()+':', os.listdir(os.getcwd()));

Instructions for updating:
non-resource variables are not supported in the long term
C:\Users\bhs89\machine-learning: ['.ipynb_checkpoints', 'cache', 'datasets_for_clustering', 'dataset_for_clustering.ipynb', 'Machine_learning', 'Team_project']
C:\Users\bhs89\machine-learning\Team_project: ['-p', '.ipynb_checkpoints', 'baseline_hong-Copy1.ipynb', 'baseline_hong.ipynb', 'Dementia_df_hong.ipynb', 'EDA_report_code_dementia.ipynb', 'input', 'RNN', 'RNN.ipynb', 'saved_model', 'Untitled.ipynb']


In [17]:
cols_convert = data.columns[60:66]
data_convert = data[cols_convert].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [18]:
# OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
ohe

OneHotEncoder(sparse=False)

In [19]:
input_data_y

0       CN
1       CN
2       CN
3       CN
4       CN
        ..
9322    CN
9323    CN
9324    CN
9325    CN
9326    CN
Name: DIAG_NM, Length: 9327, dtype: object

In [20]:
input_data_y.shape
input_data_y.values.reshape(-1,1).shape

(9327, 1)

In [21]:
ohe.fit(input_data_y.values.reshape(-1,1))
one_hot_encoded = ohe.transform(input_data_y.values.reshape(-1,1))
one_hot_encoded

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [22]:
print(type(ohe.categories_))
ohe.categories_

<class 'list'>


[array(['CN', 'Dem', 'MCI'], dtype=object)]

In [23]:
ohe_df = pd.DataFrame(one_hot_encoded, columns=ohe.categories_[0])
ohe_df

,CN,Dem,MCI
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9322,1.0,0.0,0.0
9323,1.0,0.0,0.0
9324,1.0,0.0,0.0
9325,1.0,0.0,0.0


In [24]:
input_data_y  = ohe_df.copy()
input_data_y

,CN,Dem,MCI
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
9322,1.0,0.0,0.0
9323,1.0,0.0,0.0
9324,1.0,0.0,0.0
9325,1.0,0.0,0.0


In [26]:
X1_train, X1_test, y1_train, y1_test = train_test_split(input_data_X1, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)

In [27]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.2, shuffle=True, stratify=y1_train, random_state=2022)

In [28]:
print('훈련용  activity_class_5min: {}'.format(len(X1_train)))
print('검증용  activity_class_5min: {}'.format(len(X1_val)))
print('테스트용 activity_class_5min : {}'.format(len(X1_test)))
# num_classes = len(set(y1_train))
# print('label: {}'.format(num_classes))

훈련용  activity_class_5min: 5968
검증용  activity_class_5min: 1493
테스트용 activity_class_5min : 1866


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Activation, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.layers import Bidirectional
from keras import optimizers
from sklearn.metrics import accuracy_score
from keras.layers import Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

In [30]:
X2_train, X2_test, y2_train, y2_test = train_test_split(input_data_X2, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X3_train, X3_test, y3_train, y3_test = train_test_split(input_data_X3, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X4_train, X4_test, y4_train, y4_test = train_test_split(input_data_X4, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)
X5_train, X5_test, y5_train, y5_test = train_test_split(input_data_X5, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=2022)

In [31]:
print("X3 :" ,X3_train.shape)
print("y3 :" ,y3_train.shape)
print("X4 :" ,X4_train.shape)
print("y4 :" ,y4_train.shape)
print("X5 :" ,X5_train.shape)
print("y5 :" ,y5_train.shape)

X3 : (7461, 181)
y3 : (7461, 3)
X4 : (7461, 180)
y4 : (7461, 3)
X5 : (7461, 181)
y5 : (7461, 3)


In [32]:
X3_train =  X3_train.to_numpy()
X3_train

array([[ 0., 70., 74., ...,  0.,  0.,  0.],
       [ 0.,  0., 74., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 87., 87., ...,  0.,  0.,  0.],
       [64., 60., 59., ...,  0.,  0.,  0.]])

In [33]:
X3_test =  X3_test.to_numpy()
X3_test

array([[ 0., 55., 55., ...,  0.,  0.,  0.],
       [ 0.,  0., 66., ...,  0.,  0.,  0.],
       [ 0., 66., 66., ...,  0.,  0.,  0.],
       ...,
       [ 0., 78., 74., ...,  0.,  0.,  0.],
       [ 0., 50., 50., ...,  0.,  0.,  0.],
       [62., 62., 63., ...,  0.,  0.,  0.]])

In [34]:
X3_train = X3_train.reshape(X3_train.shape[0],X3_train.shape[1],1)
#X3_val = X3_val.reshape(X3_val.shape[0],X3_val.shape[1],1)
X3_test = X3_test.reshape(X3_test.shape[0],X3_test.shape[1],1)

In [35]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(3, input_shape = (181,1), return_sequences = False)) #input shape에 열 개수 넣어주면 됨
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) #label encoding 했기때문에  'sparse_categorical_crossentropy' 사용
    
    return model

In [36]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)

Train on 7461 samples
Epoch 1/10
7461/7461 [==============================] - 0s 51us/sample - loss: 0.9750 - acc: 0.5905
Epoch 2/10
7461/7461 [==============================] - 0s 44us/sample - loss: 0.8650 - acc: 0.6397
Epoch 3/10
7461/7461 [==============================] - 0s 47us/sample - loss: 0.8421 - acc: 0.6397
Epoch 4/10
7461/7461 [==============================] - 0s 48us/sample - loss: 0.8352 - acc: 0.6393
Epoch 5/10
7461/7461 [==============================] - 0s 41us/sample - loss: 0.8308 - acc: 0.6396
Epoch 6/10
7461/7461 [==============================] - 0s 41us/sample - loss: 0.8281 - acc: 0.6396
Epoch 7/10
7461/7461 [==============================] - 0s 44us/sample - loss: 0.8261 - acc: 0.6393
Epoch 8/10
7461/7461 [==============================] - 0s 43us/sample - loss: 0.8215 - acc: 0.6381
Epoch 9/10
7461/7461 [==============================] - 0s 48us/sample - loss: 0.8156 - acc: 0.6376
Epoch 10/10
7461/7461 [==============================] - 0s 46us/sample - loss

In [39]:
len(input_data_y_label)

9327

In [41]:
len(y3_pred)

1866

In [44]:
len(y3_test_Label)

1866

In [45]:
print("accouracy : ", accuracy_score(y3_pred,y3_test_Label))

accouracy :  0.6339764201500536
